# LLM - RAG (대규모 언어 모델 - 검색 증강 생성)

# 개요
이 데모는 GPT-4 모델을 검색 증강 생성(RAG) 설정에서 사용하는 방법을 보여줍니다. \
모델은 자연어 질문을 분석하고, 지식 기반에서 관련 정보를 검색한 후, 검색된 정보를 기반으로 답변을 제공합니다.



In [23]:
import re
import requests
import sys
import os
from openai import AzureOpenAI
import tiktoken
from dotenv import load_dotenv
load_dotenv()

client = AzureOpenAI(
  azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"), 
  api_key=os.getenv("AZURE_OPENAI_KEY"),  
  api_version="2024-02-15-preview"
)

CHAT_COMPLETIONS_MODEL = os.getenv('AZURE_OPENAI_DEPLOYMENT_NAME')

### 자연어 질문으로 시작하기

In [2]:
input = "2025년 한국 대선에서 어느 대통령 후보가 당선될 가능성이 높은가요? 그 이유를 설명하세요."

### 1단계: GPT-4o: 이 질문에 답하기 위해 무엇이 필요한가요?

In [3]:
system_prompt=f'''다음 질문에 가능한 한 최선을 다해 답하세요. 다음 도구에 액세스할 수 있습니다:

웹 검색: 웹을 사용하여 정보를 찾습니다.

아래 형식을 엄격하게 따라야 합니다:

질문: 반드시 답해야 하는 입력 질문
생각: 무엇을 해야 할지 항상 생각해야 합니다.
행동: 취해야 할 행동, [조사] 중 하나이어야 합니다.
행동 입력: 행동에 대한 입력

시작!
'''
user_prompt = f'''
Question: {input}
Thought:'''

In [4]:
response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":system_prompt},
                {"role":"user","content": user_prompt,}],
        max_tokens=400,)

print(response.choices[0].message.content)

질문: 2025년 한국 대선에서 어느 대통령 후보가 당선될 가능성이 높은가요? 그 이유를 설명하세요.
생각: 현재는 2025년 대선에 관한 최신 정보가 없으므로, 최신 뉴스와 전문가들의 의견을 조사해야 합니다.
행동: 조사
행동 입력: 2025년 한국 대선 후보와 선거 예측 관련 최신 뉴스나 전문가 의견 검색.


### 2단계: 더 많은 세부 정보를 위해 웹 검색


계속하기 전에 구독에서 검색 서비스를 생성하세요 [검색 리소스 생성](https://serpapi.com). \
이 샘플은 쿼리를 사용하여 웹 검색 API에 호출을 수행하고 관련 웹 검색 결과를 반환합니다.

In [ ]:


import serpapi
import json

params = {
  "engine": "google",
  "q": "2025년 한국 대선에서 어느 대통령 후보가 당선될 가능성이 높은가요? 그 이유를 설명하세요.",
  "api_key": os.getenv("SERP_API_KEY"),
}
print()

response = serpapi.search(params)

# response 객체를 JSON 문자열로 변환 후, ensure_ascii=False로 한글이 깨지지 않게 출력
print(json.dumps(response.data, ensure_ascii=False, indent=2))

context = ''
organic_results = response["organic_results"]

for i, result in enumerate(organic_results):
    title = result.get("title")
    link = result.get("link")
    snippet = result.get("snippet")
    # Ensure the title is printed as a proper Korean string
    if isinstance(snippet, bytes):
        snippet = snippet.decode('utf-8')
    if isinstance(title, bytes):
        title = title.decode('utf-8')
    print(f"Result {i+1}:")
    print(f"Title: {title}")
    print(f"Link: {link}")
    print(f"Snippet:{snippet}")
    context += snippet + '\n'


{
  "search_metadata": {
    "id": "681c048a124815de3d2dcf34",
    "status": "Success",
    "json_endpoint": "https://serpapi.com/searches/d29be1be5bcaa72d/681c048a124815de3d2dcf34.json",
    "created_at": "2025-05-08 01:10:34 UTC",
    "processed_at": "2025-05-08 01:10:34 UTC",
    "google_url": "https://www.google.com/search?q=2025%EB%85%84+%ED%95%9C%EA%B5%AD+%EB%8C%80%EC%84%A0%EC%97%90%EC%84%9C+%EC%96%B4%EB%8A%90+%EB%8C%80%ED%86%B5%EB%A0%B9+%ED%9B%84%EB%B3%B4%EA%B0%80+%EB%8B%B9%EC%84%A0%EB%90%A0+%EA%B0%80%EB%8A%A5%EC%84%B1%EC%9D%B4+%EB%86%92%EC%9D%80%EA%B0%80%EC%9A%94%3F+%EA%B7%B8+%EC%9D%B4%EC%9C%A0%EB%A5%BC+%EC%84%A4%EB%AA%85%ED%95%98%EC%84%B8%EC%9A%94.&oq=2025%EB%85%84+%ED%95%9C%EA%B5%AD+%EB%8C%80%EC%84%A0%EC%97%90%EC%84%9C+%EC%96%B4%EB%8A%90+%EB%8C%80%ED%86%B5%EB%A0%B9+%ED%9B%84%EB%B3%B4%EA%B0%80+%EB%8B%B9%EC%84%A0%EB%90%A0+%EA%B0%80%EB%8A%A5%EC%84%B1%EC%9D%B4+%EB%86%92%EC%9D%80%EA%B0%80%EC%9A%94%3F+%EA%B7%B8+%EC%9D%B4%EC%9C%A0%EB%A5%BC+%EC%84%A4%EB%AA%85%ED%95%98%EC%84%B8%EC%9A

### 추가 처리를 위한 수집된 정보

In [18]:
print(context)

국민의힘은 22일 1차 컷오프 결과 대선 경선 후보가 김문수 전 고용노동부 장관, 안철수 의원, 한동훈 전 국민의힘 대표, 홍준표 전 대구시장 등 4명 ...
조사결과 요약 투표 참여 의향 '반드시 투표할 것' 적극적 투표층 82% 더불어민주당 후보 적합도 -이재명 44%, 김동연 8%, 김경수 3%, 김두관 2% 등의 ...
2025년 조기 대선, 2017년과는 다르다? '19대 대선' 文 41.08% 당선… 나머지 후보들 58.37% 양당 구도 짙어진 2025년… '표 분산' 없다?
그리고 기본소득당과 사회민주당이 후보자를 추천하지 아니하기로 선언하면서, 진보당의 후보 등록 여부에 따라 원외정당·무소속 후보자들의 기호 배정은 5번-6번에서 시작 ...
제21대 대통령선거 정책토론회 - 2025. 05. 02.(금) 10:00 「공직선거법」제82조의3(선거방송토론위원회 주관 정책토론회)에 따라 개최하는 제21대 ...
국민의힘과 더불어민주당이 21대 대선 경선 후보 등록을 마감하고 최종 후보 선출을 위한 본격적인 레이스에 들어갔다. 국민의힘은 16일 대통령선거 1 ...
민주당 대선 후보인 카멀라 해리스(왼쪽) 부통령, 공화당 후보인 도널드 트럼프 전 대통령. / 로이터. 약 10일 후 치러지는 미국 대통령 선거에 세계 ...
깃발이 분명하면 결국은 이기게 되어 있다.” 노무현 대통령이 남긴 말이다. 노무현 정부 청와대 대변인을 맡았던 윤태영의 『기록』에 나와 있다. 정치의 ...



In [19]:
system_prompt=f'''Answer the following questions as best you can. Use the context provided below:
{context}

Begin!
'''
user_prompt = f'''
Question: {input}\
Answer:'''

### 3단계: GPT-4o: 이 질문에 답하기 위해 무엇을 해야 하나요? (다시)

In [20]:
response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":system_prompt},
                {"role":"user","content": user_prompt,}],
        max_tokens=400,)

print(response.choices[0].message.content)

2025년 한국 대선에서는 후보들이 아직 확정되지 않았고, 선거 결과도 예측할 수 없는 많은 변수들이 있기 때문에 특정 후보가 당선될 가능성을 확정적으로 말하기는 어렵습니다. 다만, 여론조사와 정치적 분위기, 정당의 지지도 등을 고려할 때 몇 가지 점을 고려할 수 있습니다.

1. **여론조사**: 적합도 조사에서 이재명 후보가 44%로 높은 지지를 받고 있으며, 더불어민주당의 주요 후보로 활동하고 있습니다. 이재명 후보가 현재 높은 지지율을 유지한다면 경쟁력 있는 후보로 볼 수 있습니다.

2. **정당 구도**: 국민의힘과 더불어민주당의 양당 구도가 짙어지는 상황에서 두 당의 후보들은 유력할 가능성이 높습니다. 각 정당의 최종 후보로 자리잡는다면 당의 지지 기반을 최대한 활용할 수 있게 됩니다.

3. **정치 이슈와 사회 분위기**: 대선 후보들은 정책 토론회 등을 통해 국민들에게 정책 비전을 제시하며 자신의 입지를 강화할 것입니다. 국민의 주요 관심사에 적합한 답변을 제시하는 후보가 더 많은 지지를 받을 가능성이 높습니다.

이러한 요소들을 종합적으로 고려할 때, 특정 후보의 당선 가능성을 정확히 예측하는 것은 어렵지만, 현재로서는 이재명 후보가 강력한 경쟁력을 가진 후보 중 하나로 보입니다. 그러나 선거까지 많은 시간이 남아 있어 정치적 변화와 새로운 후보 등장 등 변수들이 다수 존재하므로 상황은 계속적으로 변동할 수 있습니다.


### GPT-4o: 이제 질문에 대한 최종 답변을 얻었습니다.

#### 탐색할 사용 사례
1. **지식 관리** \
내부 문서, 매뉴얼 및 정책에서 정보를 검색하고 제공하여 기업 지식에 대한 접근성을 향상시킵니다.

2. **비즈니스 인텔리전스 및 분석** \
시장 분석가가 다양한 소스에서 관련 시장 데이터, 보고서 및 통찰력을 검색할 수 있도록 지원합니다.

3. **법률 및 준수** \
법률 전문가가 쿼리에 따라 관련 판례, 법령 및 법적 선례를 검색할 수 있도록 지원합니다.